In [1]:
import os
import shutil

import re
import time
import pandas as pd
import numpy as np

import pprint
from ast import literal_eval
#from gensim.models import Phrases
#import gensim
#from gensim.utils import simple_preprocess
#import gensim.corpora as corpora
# NLTK Stop words
from ast import literal_eval
import spacy
from nltk.corpus import stopwords
#stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use','a','about', 'above', 'across'])

NewCorpus = pd.read_excel('CorpusBeta.xlsx',header=0)
NewCorpus['From-->Too'] = NewCorpus['From-->Too'].apply(literal_eval)
NewCorpus['Method Used'] = NewCorpus['Method Used'].apply(literal_eval)

print('la taille de la corpus : '+str(len(NewCorpus)))
NewCorpus.head()

la taille de la corpus : 2014


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,"[persian, arabic, english]",[statistical],[]
1,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,"[arabic, english]",[statistical],[]
2,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,"[hindi, english]",[statistical],[]
3,Combining MEDLINE and publisher data to create...,Background: Most of the institutional and rese...,Biomedical domain; Multilingual corpus generat...,https://www.scopus.com/inward/record.uri?eid=2...,2013,14.0,NaN,NaN,1,10.1186/1471-2105-14-146,"Jimeno Yepes A., Prieur-Gaston É., Névéol A.",Article,Scopus,"[spanish, french, english]",[statistical],[]
4,Syntax-based Chinese-Vietnamese tree-to-tree s...,"Because of the scarcity of bilingual corpora, ...",Chinese-Vietnamese; Linguistic features; Pruni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3314938,"Gao S., Huang J., Xue M., Yu Z., Wang Z., Zhan...",Article,Scopus,"[vietnamese, chinese]",[statistical],[]


In [2]:
# keep only articles that have more than two languages 
NewCorpus = NewCorpus.loc[NewCorpus['From-->Too'].apply(lambda x: len(x) >= 1)]
NewCorpus.reset_index(drop=True, inplace=True)
print('the length  of the corpus of articles that treat a language or more : '+str(len(NewCorpus)))
NewCorpus.head() 

the length  of the corpus of articles that treat a language or more : 1268


,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,"[persian, arabic, english]",[statistical],[]
1,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,"[arabic, english]",[statistical],[]
2,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,"[hindi, english]",[statistical],[]
3,Combining MEDLINE and publisher data to create...,Background: Most of the institutional and rese...,Biomedical domain; Multilingual corpus generat...,https://www.scopus.com/inward/record.uri?eid=2...,2013,14.0,NaN,NaN,1,10.1186/1471-2105-14-146,"Jimeno Yepes A., Prieur-Gaston É., Névéol A.",Article,Scopus,"[spanish, french, english]",[statistical],[]
4,Syntax-based Chinese-Vietnamese tree-to-tree s...,"Because of the scarcity of bilingual corpora, ...",Chinese-Vietnamese; Linguistic features; Pruni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3314938,"Gao S., Huang J., Xue M., Yu Z., Wang Z., Zhan...",Article,Scopus,"[vietnamese, chinese]",[statistical],[]


In [3]:
len(NewCorpus)

1268

In [4]:
ToLanguages_Translation = []

#Remove All words that could mess our classification
ToLanguages = ["to mandarin","to chinese","to spanish","to Jalaa","to Maori","to Ryukyuan","to Maroon","to Mayan",
               "to Sranan","to Nahuatl","to mam","to Patois","to Xinca","to Quiche","to Kekchi","to Cakchiquel",
               "to Nahua","to Carib","to Amerindian","to Garifuna","to Mesrop","to Khenkha","to Tshanglakha",
               "to Lhotsamkha","to Nepalese","to Waray","to Pampango","to Pangasinense","to Bicol","to Aramaic",
               "to Circassian","to Tajik","to Baluchi","to Khalkha","to mongol","to Dhivehi","to Maldivian",
               "to Altaic","to Kyrgyz","to Bahasa","to Hangungmal","to Qazaq","to Turcoman","to Assyrian",
               "to Yiddish","to Judeo","to Ladino","to Tuvaluan","to Kejia","to Tibetan","to Tibet","to Mongolian",
               "to Bhutanese","to Darussalam","to Dzongkha","to Farsi","to Turkic","to Azeri","to Bislama",
               "to Bichelama","to Ulithian","to Pitcairnese","to Pisin","to Motu","to Sonsoralese","to Angaur",
               "to Carolinian","to Niuean","to Tongan","to Polynesian","to Melanesian","to Kapingamarangi",
               "to Tahitian","to Nukuoro","to Woleaian","to Trukese","to Pohnpeian","to Kosrean","to Yapese",
               "to Kiribati","to Chamorro","to Ibibio","to Fijian","to Welsh","to gaelic","to Samoan ","to Nordic",
               "to Monegasque","to Gagauz","to Lahnda","to Llanito ","to Sami","to Castilian","to Zande","to Chadic",
               "to Sena","to Volta","to twi","to Wolof","to Umbundu","to Tsoa","to Tshivenda","to Tshiluba","to Tshwa",
               "to kua","to Tonga","to Swazi","to Seychellois","to Mauritian","to Noon","to Nambya","to Ndau",
               "to Kimbundu","to Sotho","to Luganda","to Lingala","to Kituba","to Kongo","to fon","to Fulani",
               "to Kikongo","to Verdean","to Abron","to Afar","to Dangme","to Berber","to Khoekhoe","to Seychelles",
               "to Comorian","to Sesotho","to Ndebele","to Sepedi","to Setswana","to Tswana","to Sindebele",
               "to Tigrinya","to Swati","to Swahili","to Tsonga","to Venda","to Soranî","to Mimi","to Ga","to Yeni",
               "to Shabo","to Weyto","to Wutana","to Mpra","to Oropom","to Ongota","to Oblo","to Bangi","to Meyobe",
               "to Mawa","to Luo","to Kujarge","to Lufu","to Laal","to Lezgi","to Hadza","to Irimba","to Gumuz",
               "to Gomba","to Bayot","to Ega","to Dompo","to Ingush","to Kabardian","to Tatar","to Esperanto",
               "to Chechen","to Chuvash","to Dargwa","to Bashkir","to Riksmål","to Latin","to Riksmal","to Galician",
               "to Catalan","to Tamazight","to Sorani","to Slovene","to Afrikaans","to Armenian","to Slovak",
               "to Danish","to Norwegian","to Icelandic","to Finnish","to Dhundhari","to Fuzhou","to Balochi",
               "to Xhosa","to Belarusian","to Mossi","to Ilocano","to Shona","to Hiligaynon","to Ilonggo","to Hmong",
               "to Swedish","to Quechua","to Kirundi","to Haitian","to Creole","to Zhuang","to Akan","to Chewa",
               "to Haryanvi","to Marwari","to Madurese","to Turkmen","to Serbo","to Sinhala","to Malagasy",
               "to Croatian","to Fula","to Gan","to Visayan","to Awadhi","to Oromo","to Sundanese","to Teochew",
               "to Min","to Cantonese","to Hokkien","to Shanghainese","to english","to Bodo","to Filipino","to Dogri",
               "to Santhali","to Oriya","to Sanskrit","to hindi","to Manipuri","to Konkani","to Kashmiri",
               "to hindustani","to bengali","to portuguese","to russian","to japanese","to western","to punjabi",
               "to marathi","to Telugu","to Wu","to Turkish","to Korean","to French","to German","to Vietnamese",
               "to Tamil","to Yue","to Urdu","to Javanese","to Italian","to Egyptian","to Arabic","to Gujarati",
               "to Iranian","to Persian","to Bhojpuri","to Minnan","to nan","to banlam","to Hakka","to Jin","to Hausa",
               "to Kannada","to Indonesian","to Malay","to Polish","to Yoruba","to Xiang","to Malayalam","to Odia",
               "to Maithili","to Burmese","to Sunda","to Sudanese","to Algerian","to Moroccan","to Ukrainian","to Igbo",
               "to Uzbek","to Sindhi","to Romanian","to Tagalog","to Dutch","to Amharic","to Putonghua","to Pashto",
               "to Magahi","to Thai","to Saraiki","to Khmer","to Chhattisgarhi","to Somali","to Malaysian","to Malay",
               "to Cebuano","to Nepali","to Mesopotamian","to Avar","to Assamese","to Sinhalese","to Kurdish","to Hejazi",
               "to Nigerian","to Fulfulde","to Bavarian","to Azerbaijani","to Greek","to Chittagonian","to Kazakh",
               "to Deccan","to Hungarian","to Kinyarwanda","to Zulu","to Levantine","to Tunisian","to Sanaani","to MinBei",
               "to Bei","to Rundi","to Czech","to Taʽizzi-Adeni","to Uyghur","to MinDong","to Dong","to Sylheti",
               "into mandarin","into chinese","into spanish","into Jalaa","into Maori","into Ryukyuan","into Maroon",
               "into Mayan","into Sranan","into Nahuatl","into mam","into Patois","into Xinca","into Quiche","into Kekchi",
               "into Cakchiquel","into Nahua","into Carib","into Amerindian","into Garifuna","into Mesrop","into Khenkha",
               "into Tshanglakha","into Lhotsamkha","into Nepalese","into Waray","into Pampango","into Pangasinense",
               "into Bicol","into Aramaic","into Circassian","into Tajik","into Baluchi","into Khalkha","into mongol",
               "into Dhivehi","into Maldivian","into Altaic","into Kyrgyz","into Bahasa","into Hangungmal","into Qazaq",
               "into Turcoman","into Assyrian","into Yiddish","into Judeo","into Ladino","into Tuvaluan","into Kejia",
               "into Tibetan","into Tibet","into Mongolian","into Bhutanese","into Darussalam","into Dzongkha","into Farsi",
               "into Turkic","into Azeri","into Bislama","into Bichelama","into Ulithian","into Pitcairnese","into Pisin",
               "into Motu","into Sonsoralese","into Angaur","into Carolinian","into Niuean","into Tongan","into Polynesian",
               "into Melanesian","into Kapingamarangi","into Tahitian","into Nukuoro","into Woleaian","into Trukese",
               "into Pohnpeian","into Kosrean","into Yapese","into Kiribati","into Chamorro","into Ibibio","into Fijian",
               "into Welsh","into gaelic","into Samoan ","into Nordic","into Monegasque","into Gagauz","into Lahnda",
               "into Llanito ","into Sami","into Castilian","into Zande","into Chadic","into Sena","into Volta","into twi",
               "into Wolof","into Umbundu","into Tsoa","into Tshivenda","into Tshiluba","into Tshwa","into kua","into Tonga",
               "into Swazi","into Seychellois","into Mauritian","into Noon","into Nambya","into Ndau","into Kimbundu",
               "into Sotho","into Luganda","into Lingala","into Kituba","into Kongo","into fon","into Fulani","into Kikongo",
               "into Verdean","into Abron","into Afar","into Dangme","into Berber","into Khoekhoe","into Seychelles",
               "into Comorian","into Sesotho","into Ndebele","into Sepedi","into Setswana","into Tswana","into Sindebele",
               "into Tigrinya","into Swati","into Swahili","into Tsonga","into Venda","into Soranî","into Mimi","into Ga",
               "into Yeni","into Shabo","into Weyto","into Wutana","into Mpra","into Oropom","into Ongota","into Oblo",
               "into Bangi","into Meyobe","into Mawa","into Luo","into Kujarge","into Lufu","into Laal","into Lezgi",
               "into Hadza","into Irimba","into Gumuz","into Gomba","into Bayot","into Ega","into Dompo","into Ingush",
               "into Kabardian","into Tatar","into Esperanto","into Chechen","into Chuvash","into Dargwa","into Bashkir",
               "into Riksmål","into Latin","into Riksmal","into Galician","into Catalan","into Tamazight","into Sorani",
               "into Slovene","into Afrikaans","into Armenian","into Slovak","into Danish","into Norwegian","into Icelandic",
               "into Finnish","into Dhundhari","into Fuzhou","into Balochi","into Xhosa","into Belarusian","into Mossi",
               "into Ilocano","into Shona","into Hiligaynon","into Ilonggo","into Hmong","into Swedish","into Quechua",
               "into Kirundi","into Haitian","into Creole","into Zhuang","into Akan","into Chewa","into Haryanvi",
               "into Marwari","into Madurese","into Turkmen","into Serbo","into Sinhala","into Malagasy","into Croatian",
               "into Fula","into Gan","into Visayan","into Awadhi","into Oromo","into Sundanese","into Teochew","into Min",
               "into Cantonese","into Hokkien","into Shanghainese","into english","into Bodo","into Filipino","into Dogri",
               "into Santhali","into Oriya","into Sanskrit","into hindi","into Manipuri","into Konkani","into Kashmiri",
               "into hindustani","into bengali","into portuguese","into russian","into japanese","into western","into punjabi",
               "into marathi","into Telugu","into Wu","into Turkish","into Korean","into French","into German","into Vietnamese",
               "into Tamil","into Yue","into Urdu","into Javanese","into Italian","into Egyptian","into Arabic","into Gujarati",
               "into Iranian","into Persian","into Bhojpuri","into Minnan","into nan","into banlam","into Hakka","into Jin",
               "into Hausa","into Kannada","into Indonesian","into Malay","into Polish","into Yoruba","into Xiang","into Malayalam",
               "into Odia","into Maithili","into Burmese","into Sunda","into Sudanese","into Algerian","into Moroccan",
               "into Ukrainian","into Igbo","into Uzbek","into Sindhi","into Romanian","into Tagalog","into Dutch","into Amharic",
               "into Putonghua","into Pashto","into Magahi","into Thai","into Saraiki","into Khmer","into Chhattisgarhi",
               "into Somali","into Malaysian","into Malay","into Cebuano","into Nepali","into Mesopotamian","into Avar",
               "into Assamese","into Sinhalese","into Kurdish","into Hejazi","into Nigerian","into Fulfulde","into Bavarian",
               "into Azerbaijani","into Greek","into Chittagonian","into Kazakh","into Deccan","into Hungarian","into Kinyarwanda",
               "into Zulu","into Levantine","into Tunisian","into Sanaani","into MinBei","into Bei","into Rundi","into Czech",
               "into Taʽizzi-Adeni","into Uyghur","into MinDong","into Dong","into Sylheti"]
ToLang = [word.lower() for word in ToLanguages]

NewCorpus['Titles & Keywords'] = NewCorpus['Title'] +' '+ NewCorpus['Author Keywords'] +' '+ NewCorpus['Abstract'].str.split('©').str[0]
NewCorpus['Titles & Keywords'] = NewCorpus['Titles & Keywords'].str.lower()
# Recognition for Corpus
for i in range(len(NewCorpus["Titles & Keywords"])): 
    ToLanguageIn = []
    for j in range(len(ToLang)):
        a = re.sub(r'[^\w\s]', ' ', str(NewCorpus["Titles & Keywords"][i]))
        b = r'\b' + r'\b.*\b'.join(re.escape(word) for word in ToLang[j].split(' ')) + r'\b'
        c = bool(re.search(b,a))
        if c == True:
            ToLanguageIn.append(ToLang[j])
    ToLanguages_Translation.append(ToLanguageIn)
    
NewCorpus["To Language"] = ToLanguages_Translation
print("The number of Patterns that the program can recognize :"+str(len(ToLang)/2))
NewCorpus["Titles & Keywords"][0]

The number of Patterns that the program can recognize :333.0


'matching graph, a method for extracting parallel information from comparable corpora and arabic languages; comparable corpora; english; generative model; information extraction; natural language processing; parallel fragments; persian; statistical machine translation comparable corpora are valuable alternatives for the expensive parallel corpora. they comprise informative parallel fragments that are useful resources for different natural language processing tasks. in this work, a generative model is proposed for efficient extraction of parallel fragments from a pair of comparable documents. the core of the proposed model is a graph called the matching graph. the ability of the matching graph to be trained on a small initial seed makes it a proper model for language pairs suffering from the scarce resource problem. experiments show that the matching graph performs significantly better than other recently publishedmodels.according to the experiments on english-persian and arabic-persian

In [5]:
#df["From-->Too"] = Languages_Translation 
ToNewCorpus = NewCorpus[NewCorpus.astype(str)["To Language"] != '[]']
NoNewCorpus = NewCorpus[NewCorpus.astype(str)["To Language"] == '[]']

NoNewCorpus.reset_index(drop=True, inplace=True)
ToNewCorpus.reset_index(drop=True, inplace=True)
ToNewCorpus



,Title,Abstract,Author Keywords,Link,Year,Cited by,Page start,Page end,Page count,DOI,Authors,Document Type,Source,From-->Too,Method Used,is it a survey?,Titles & Keywords,To Language
0,"Matching graph, a method for extracting parall...",Comparable corpora are valuable alternatives f...,And Arabic languages; Comparable corpora; Engl...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3329713,"Bakhshaei S., Safabakhsh R., Khadivi S.",Article,Scopus,"[persian, arabic, english]",[statistical],[],"matching graph, a method for extracting parall...","[to english, to arabic, to persian]"
1,Machine translation errors: English and Iraqi ...,Errors in machine translations of English-Iraq...,Arabic; English; Error analysis; Evaluation; S...,https://www.scopus.com/inward/record.uri?eid=2...,2011,1.0,NaN,NaN,1,10.1145/1929908.1929910,"Condon S., Parvaz D., Aberdeen J., Doran C., F...",Article,Scopus,"[arabic, english]",[statistical],[],machine translation errors: english and iraqi ...,"[to english, to arabic, into english, into ara..."
2,An automatic and a machine-assisted method to ...,Two different methods of corpus cleaning are p...,Bilingual corpus cleaning; Statistical machine...,https://www.scopus.com/inward/record.uri?eid=2...,2019,NaN,NaN,NaN,1,10.1145/3342351,"Srivastava J., Sanyal S., Srivastava A.K.",Article,Scopus,"[hindi, english]",[statistical],[],an automatic and a machine-assisted method to ...,"[to english, to hindi]"
3,Combining MEDLINE and publisher data to create...,Background: Most of the institutional and rese...,Biomedical domain; Multilingual corpus generat...,https://www.scopus.com/inward/record.uri?eid=2...,2013,14.0,NaN,NaN,1,10.1186/1471-2105-14-146,"Jimeno Yepes A., Prieur-Gaston É., Névéol A.",Article,Scopus,"[spanish, french, english]",[statistical],[],combining medline and publisher data to create...,"[to spanish, to english, to french]"
4,Syntax-based Chinese-Vietnamese tree-to-tree s...,"Because of the scarcity of bilingual corpora, ...",Chinese-Vietnamese; Linguistic features; Pruni...,https://www.scopus.com/inward/record.uri?eid=2...,2019,1.0,NaN,NaN,1,10.1145/3314938,"Gao S., Huang J., Xue M., Yu Z., Wang Z., Zhan...",Article,Scopus,"[vietnamese, chinese]",[statistical],[],syntax-based chinese-vietnamese tree-to-tree s...,"[to chinese, to vietnamese, into chinese, into..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,Towards a lexicographic approach to lexical tr...,The translation of lexical items is still a fo...,German; Lexical clues; Lexical transfer; Lexic...,https://www.scopus.com/inward/record.uri?eid=2...,2001,6.0,21,87,67,10.1023/A:1013136005350,"Mel?uk I., Wanner L.",Article,Scopus,"[russian, german]",[],[],towards a lexicographic approach to lexical tr...,"[to russian, to german]"
1110,Human evaluation of three machine translation ...,This article aims to compare three machine tra...,Automatic metrics; Human evaluation; Machine t...,https://www.scopus.com/inward/record.uri?eid=2...,2021,NaN,97,121,25,10.35869/VIAL.V0I18.3366,"Fernández-Torné A., Matamala A.",Article,Scopus,"[spanish, german]",[],[],human evaluation of three machine translation ...,"[to spanish, to german, into german]"
1111,The impact of post-editing and machine transla...,This article presents the results of a study i...,Creativity; Literary translation; Machine tran...,https://www.scopus.com/inward/record.uri?eid=2...,2020,NaN,255,282,28,10.1075/ts.20035.gue,"Guerberof-Arenas A., Toral A.",Article,Scopus,"[catalan, english]",[],[],the impact of post-editing and machine transla...,"[to catalan, into catalan]"
1112,Uyghur-Chinese translation disambiguation meth...,This thesis studies the disambiguation method ...,Knowledge automatic-acquisition; Uyghur-chines...,https://www.scopus.com/inward/record.uri?eid=2...,2015,NaN,NaN,NaN,6,NaN,"Ge R., Yong Y., Chun X.",Article,Scopus,"[uyghur, chinese]",[],[],uyghur-chinese translation disambiguation meth...,"[to chinese, 

In [6]:
NoNewCorpus['Titles & Keywords'][0]

'implications of english as a pivot language in arabic-hindi machine translation arabic; hindi; machine translation; pivot languages; statistical machine translation in this paper, we have explored a pivot-based approach in the development of machine translation system whose parallel corpus is not available. for our study, we have taken arabic-hindi as the language pair for development of mt system and english as the pivot language. we have developed 4 mt systems using this approach. these 4 mt systems work on different methodologies. among them hierarchical phrase-based machine translation system produced better results. '

In [20]:
from re import search
#df[df.astype(str)["From --> Too"] != '[]']
#df_ar = dfof[dfof["From-->Too"].str.contains(r"arabic")]
df_ar = dfof[pd.DataFrame(dfof["From-->Too"].tolist()).isin(['arabic']).any(1).values]
df_en = dfof[pd.DataFrame(dfof["From-->Too"].tolist()).isin(['english']).any(1).values]
df_fr = dfof[pd.DataFrame(dfof["From-->Too"].tolist()).isin(['french']).any(1).values]
print("################################################################################")
print("################################################################################")
print("the number of articles that talk about machine translation and do not mention a specific language :"+str(len(dfif)))
print("################################################################################")
print("################################################################################")
print("the totale number of articles that talks about Machine Translation (From --> to) :"+str(len(dfof)))
print("################################################################################")
print("################################################################################")
print("the number of articles that talks about Arabic Machine Translation :"+str(len(df_ar)))
print("################################################################################")
print("################################################################################")
print("the number of articles that talks about English Machine Translation :"+str(len(df_en)))
print("################################################################################")
print("################################################################################")
print("the number of articles that talks about French Machine Translation :"+str(len(df_fr)))
print("################################################################################")
print("################################################################################")

################################################################################
################################################################################
the number of articles that talk about machine translation and do not mention a specific language :179
################################################################################
################################################################################
the totale number of articles that talks about Machine Translation (From --> to) :391
################################################################################
################################################################################
the number of articles that talks about Arabic Machine Translation :25
################################################################################
################################################################################
the number of articles that talks about English Machine Translation :260
###################

In [21]:
df_ar.to_excel("rest\MT_ar.xlsx", index=False)
df_en.to_excel("rest\MT_en.xlsx", index=False)
df_fr.to_excel("rest\MT_fr.xlsx", index=False)
dfof.to_excel("rest\MT_dfof.xlsx", index=False)
dfif.to_excel("rest\MT_dfif.xlsx", index=False)
len(dfof)+len(dfif)

570